# 1D Variability hypothesis testing for HBEC IFN experiment

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from pybedtools import BedTool
import pickle as pkl
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.6-py3.8.egg')
sys.path.append('/home/ssm-user/Github/misc-seq/miscseq/')
import encode
import memento

In [3]:
data_path = '/data_volume/memento/hbec/'

### Read the processed RNA data

Focus on the club and bc/club cells and type I interferons for now.

Encode the timestamps to integers.

In [4]:
adata = sc.read(data_path + 'HBEC_type_I_filtered_counts_deep.h5ad')

In [5]:
converter = {'basal/club':'BC', 'basal':'B', 'ciliated':'C', 'goblet':'G', 'ionocyte/tuft':'IT', 'neuroendo':'N'}

In [6]:
adata.obs['ct'] = adata.obs['cell_type'].apply(lambda x: converter[x])

### Setup memento

In [7]:
def assign_q(batch):
    
    if batch == 0:
        return 0.387*0.25
    elif batch == 1:
        return 0.392*0.25
    elif batch == 2:
        return 0.436*0.25
    else:
        return 0.417*0.25

In [8]:
adata.obs['q'] = adata.obs['batch'].apply(assign_q)

In [9]:
memento.setup_memento(adata, q_column='q', trim_percent=0.05)

Version 0.0.6


### Run memento for each subset, comparing beta to gamma, separate cell types

In [10]:
cts = ['C', 'B', 'BC']
tps = ['3', '6', '9', '24', '48']

stims = ['alpha', 'beta', 'gamma', 'lambda']

In [17]:
import os
done_files = os.listdir(data_path + 'binary_test_latest/')

In [ ]:
for ct in cts:
    
    for tp in tps:
            
        fname = '{}_{}_beta_vs_gamma.h5ad'.format(ct, tp)

        if fname in done_files:
            print('Skipping', fname)
            continue

        print('starting', ct, tp)

        adata_stim = adata.copy()[
            adata.obs.ct.isin([ct]) & \
            adata.obs.stim.isin(['beta', 'gamma']) & \
            adata.obs.time.isin([tp]), :].copy()
        stim_converter={'gamma':0, 'beta':1}
        adata_stim.obs['stim_indicator'] = adata_stim.obs['stim'].apply(lambda x: stim_converter[x])

        memento.create_groups(adata_stim, label_columns=['stim_indicator', 'donor'])
        memento.compute_1d_moments(adata_stim, min_perc_group=.9)

        memento.ht_1d_moments(
            adata_stim, 
            formula_like='1 + stim_indicator + donor',
            treatment_col='stim_indicator', 
            num_boot=10000, 
            verbose=1,
            num_cpus=93,
            resampling='permutation',
            approx=True)

        adata_stim.write(data_path + 'binary_test_latest/{}_{}_beta_vs_gamma.h5ad'.format(ct, tp))

### Run memento for each subset, but stratify cell types

In [10]:
cts = ['C', 'B', 'BC']
tps = ['3', '6', '9', '24', '48']

stims = ['alpha', 'beta', 'gamma', 'lambda']

In [21]:
import os
done_files = os.listdir(data_path + 'binary_test_latest/')

In [22]:
for tp in tps:

    fname = 'all_ct_{}_beta_vs_gamma.h5ad'.format(tp)

    if fname in done_files:
        print('Skipping', fname)
        continue

    print('starting', ct, tp)

    adata_stim = adata.copy()[
        adata.obs.ct.isin(['C', 'BC', 'B']) & \
        adata.obs.stim.isin(['beta', 'gamma']) & \
        adata.obs.time.isin([tp]), :].copy()
    stim_converter={'gamma':0, 'beta':1}
    adata_stim.obs['stim_indicator'] = adata_stim.obs['stim'].apply(lambda x: stim_converter[x])

    memento.create_groups(adata_stim, label_columns=['stim_indicator', 'donor', 'ct'])
    memento.compute_1d_moments(adata_stim, min_perc_group=.3)

    memento.ht_1d_moments(
        adata_stim, 
        formula_like='1 + stim_indicator + donor + ct',
        treatment_col='stim_indicator', 
        num_boot=10000, 
        verbose=1,
        num_cpus=93,
        resampling='permutation',
        approx=True)

    adata_stim.write(data_path + 'binary_test_latest/all_ct_{}_beta_vs_gamma.h5ad'.format(tp))

starting BC 3


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    2.8s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    6.0s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   10.2s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   15.1s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   20.8s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   27.4s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   35.3s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   43.7s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   53.0s
[Parallel(n_j

starting BC 6


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    3.3s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    7.0s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   11.9s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   17.5s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   24.1s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   31.9s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   41.0s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   50.8s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:  1.0min
[Parallel(n_j

starting BC 9


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    3.9s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    8.7s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   14.7s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   21.8s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   29.9s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   39.5s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   50.8s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:  1.0min
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:  1.3min
[Parallel(n_j

starting BC 24


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.7s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    3.3s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    7.3s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   12.5s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   18.6s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   25.4s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   33.7s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   43.3s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   53.4s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:  1.1min
[Parallel(n_j

starting BC 48


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    3.1s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    6.9s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   11.8s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   17.7s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   24.0s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   31.8s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   40.8s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   50.4s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:  1.0min
[Parallel(n_j

### Run memento going from 3hr to 48 hr, but stratify cell types

In [13]:
cts = ['C', 'B', 'BC']
tps = ['3', '6', '9', '24', '48']

stims = ['alpha', 'beta', 'gamma', 'lambda']

In [14]:
import os
done_files = os.listdir(data_path + 'binary_test_latest/')

In [15]:
for stim in stims:

    fname = 'all_ct_{}_3_vs_48.h5ad'.format(stim)

    if fname in done_files:
        print('Skipping', fname)
        continue

    print('starting', stim)

    adata_stim = adata.copy()[
        adata.obs.ct.isin(cts) & \
        adata.obs.stim.isin(['control', stim]) & \
        adata.obs.time.isin(['3','48']), :].copy()
    time_converter={3:0, int('48'):1}
    adata_stim.obs['time_step'] = adata_stim.obs['time'].astype(int).apply(lambda x: time_converter[x])

    memento.create_groups(adata_stim, label_columns=['time_step', 'donor', 'ct'])
    memento.compute_1d_moments(adata_stim, min_perc_group=.3)

    memento.ht_1d_moments(
        adata_stim, 
        formula_like='1 + time_step + donor + ct',
        treatment_col='time_step', 
        num_boot=10000, 
        verbose=1,
        num_cpus=93,
        resampling='permutation',
        approx=True)

    adata_stim.write(data_path + 'binary_test_latest/all_ct_{}_3_vs_48.h5ad'.format(stim))

starting alpha


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    3.1s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    6.6s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   11.3s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   17.0s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   23.4s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   30.8s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   39.6s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   49.0s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   59.5s
[Parallel(n_j

starting beta


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.5s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    2.7s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    5.9s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   10.1s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   15.0s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   20.3s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   26.8s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   34.1s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   42.0s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   51.2s
[Parallel(n_j

starting gamma


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    3.2s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    6.9s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   11.9s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   17.8s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   24.4s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   32.3s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   41.7s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   51.4s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:  1.0min
[Parallel(n_j

starting lambda


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    2.9s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    6.2s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   10.5s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   15.8s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   21.6s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   28.5s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   36.3s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   45.1s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   55.2s
[Parallel(n_j